In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
import numpy as np

# Step 1: Define your custom dataset
class CustomDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        target = self.targets[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample, target

seed = 12
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Step 2: Define transformations for your custom dataset
transform = T.Compose([
        T.RandomHorizontalFlip(),
        T.RandomVerticalFlip(),
        T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.10),
        T.RandomAffine(degrees=(-30, 30), translate=(0.2, 0.2), scale=(0.8, 1.2), shear=(-10, 10)),
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])


# Step 3: Define function to get data loaders for train, validation, and test datasets
def get_data_loaders(data_dir, test_dir, batch_size):
    train_data = datasets.ImageFolder(os.path.join(data_dir), transform=transform)
    test_data = datasets.ImageFolder(os.path.join(test_dir), transform=test_transform)

    train_size = int(0.8 * len(train_data))
    val_size = len(train_data) - train_size
    train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, val_loader, test_loader

import timm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Step 4: Load pre-trained PyTorch models
model1 = torch.hub.load('pytorch/vision:v0.10.0', 'densenet169', pretrained=True)
model2 = torch.hub.load('pytorch/vision:v0.10.0', 'densenet201', pretrained=True)
model3 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model4 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model5 = timm.create_model('mobilenetv2_140', pretrained=True)
model6 = timm.create_model('efficientnet_b5', pretrained=True)

# Modify the models for binary classification
import torch
import torch.nn as nn
import torch.nn.init as init

# Assuming 'model' is your existing model
# Freeze all parameters in the existing model
for param in model1.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model1.classifier.in_features

# Modify the fc layer and add more layers for reducing val loss
model1.classifier = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model1 = model1.to(device)


import torch
import torch.nn as nn
import torch.nn.init as init

# Assuming 'model' is your existing model
# Freeze all parameters in the existing model
for param in model2.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model2.classifier.in_features

# Modify the fc layer and add more layers for reducing val loss
model2.classifier = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model2 = model2.to(device)



for param in model3.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model3.fc.in_features

# Modify the fc layer and add more layers for reducing val loss
model3.fc = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model3 = model3.to(device)

for param in model4.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model4.fc.in_features

# Modify the fc layer and add more layers for reducing val loss
model4.fc = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model4 = model4.to(device)

for param in model5.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model5.classifier.in_features

# Modify the fc layer and add more layers for reducing val loss
model5.classifier = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model5 = model5.to(device)

for param in model6.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model6.classifier.in_features

# Modify the fc layer and add more layers for reducing val loss
model6.classifier = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model6 = model6.to(device)

# Step 5: Load pre-trained PyTorch models
model1_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_DenseNet169.pth"
model2_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_DenseNet201.pth"
model3_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_ResNet101.pth"
model4_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_ResNet152.pth"
model5_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_MobileNetV2.pth"
model6_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_EfficientNet.pth"



model1.load_state_dict(torch.load(model1_path))
model2.load_state_dict(torch.load(model2_path))
model3.load_state_dict(torch.load(model3_path))
model4.load_state_dict(torch.load(model4_path))
model5.load_state_dict(torch.load(model5_path))
model6.load_state_dict(torch.load(model6_path))


# Set models to evaluation mode
model1.eval()
model2.eval()
model3.eval()
model4.eval()
model5.eval()
model6.eval()

# Step 6: Make predictions using the pre-trained models
def get_predictions(model, dataloader):
    all_predictions = []
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)  # Move inputs to the appropriate device
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())  # Convert to numpy array for stacking
    return all_predictions

# Step 7: Stack the predictions for meta-model training
train_loader, val_loader, test_loader = get_data_loaders("/kaggle/input/bavumub-shortwork/trainFinal/", "/kaggle/input/augmented-20x-10x/20x/20x/test", batch_size=128)

# Get predictions for training and validation sets
predictions_model1_train = get_predictions(model1, train_loader)
predictions_model2_train = get_predictions(model2, train_loader)
predictions_model3_train = get_predictions(model3, train_loader)
predictions_model4_train = get_predictions(model4, train_loader)
predictions_model5_train = get_predictions(model5, train_loader)
predictions_model6_train = get_predictions(model6, train_loader)

stacked_predictions_train = np.column_stack((predictions_model1_train, predictions_model2_train, predictions_model3_train,predictions_model4_train,predictions_model5_train,predictions_model6_train))

# Get predictions for validation set for training the meta-model
predictions_model1_val = get_predictions(model1, val_loader)
predictions_model2_val = get_predictions(model2, val_loader)
predictions_model3_val = get_predictions(model3, val_loader)
predictions_model4_val = get_predictions(model4, val_loader)
predictions_model5_val = get_predictions(model5, val_loader)
predictions_model6_val = get_predictions(model6, val_loader)


stacked_predictions_val = np.column_stack((predictions_model1_val, predictions_model2_val, predictions_model3_val, predictions_model4_val, predictions_model5_val,predictions_model6_val))

# Extract labels for the validation set
val_labels = []
for _, target in val_loader.dataset:
    val_labels.append(target)
val_labels = np.array(val_labels)

# Train the logistic regression meta-model
#meta_model = SVC() 
#meta_model = GradientBoostingClassifier() 
meta_model = DecisionTreeClassifier()
#meta_model = LogisticRegression(random_state=45), 
#meta_model = KNeighborsClassifier() 
#meta_model = AdaBoostClassifier() 
#meta_model = XGBClassifier()
#meta_model = GaussianNB() 
#meta_model = GradientBoostingClassifier()  
#meta_model = RandomForestClassifier()
meta_model.fit(stacked_predictions_val, val_labels)

# Step 9: Make predictions on the test dataset
test_predictions_model1 = get_predictions(model1, test_loader)
test_predictions_model2 = get_predictions(model2, test_loader)
test_predictions_model3 = get_predictions(model3, test_loader)
test_predictions_model4 = get_predictions(model4, test_loader)
test_predictions_model5 = get_predictions(model5, test_loader)
test_predictions_model6 = get_predictions(model6, test_loader)


test_stacked_predictions = np.column_stack((test_predictions_model1, test_predictions_model2, test_predictions_model3, test_predictions_model4, test_predictions_model5,test_predictions_model6))

# Extract labels for the test set
test_labels = []
for _, target in test_loader.dataset:
    test_labels.append(target)
test_labels = np.array(test_labels)

# Step 10: Use the meta-model to make predictions on the stacked test predictions
test_meta_preds = meta_model.predict(test_stacked_predictions)

# Step 11: Evaluate the accuracy of the stacked ensemble model
ensemble_accuracy = accuracy_score(test_labels, test_meta_preds)
print("Stacked Ensemble Accuracy:", ensemble_accuracy)


Hybrid CNN ELM Ensemble

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets
from sklearn.metrics import accuracy_score
import numpy as np
import timm

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class ELMClassifier:
    def __init__(self, n_hidden_units=100, n_output_neurons=2):
        self.n_hidden_units = n_hidden_units
        self.n_output_neurons = n_output_neurons

    def fit(self, X, y):
        # Normalize features to be in the range [-1, 1]
        X = 2 * (X - np.min(X)) / (np.max(X) - np.min(X)) - 1

        self.input_weights = np.random.normal(size=(X.shape[1], self.n_hidden_units))
        self.bias = np.random.normal(size=(self.n_hidden_units,))
        hidden_activations = np.dot(X, self.input_weights) + self.bias

        # Moore-Penrose pseudo-inverse to calculate output weights
        self.output_weights = np.linalg.pinv(hidden_activations).dot(y)

    def predict(self, X):
        # Normalize features to be in the range [-1, 1]
        X = 2 * (X - np.min(X)) / (np.max(X) - np.min(X)) - 1

        hidden_activations = np.dot(X, self.input_weights) + self.bias
        output = np.dot(hidden_activations, self.output_weights)
        return np.argmax(output, axis=1)

# Set random seed for reproducibility
seed = 12
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Define transformations for your custom dataset
transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.10),
    T.RandomAffine(degrees=(-30, 30), translate=(0.2, 0.2), scale=(0.8, 1.2), shear=(-10, 10)),
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Define function to get data loaders for train, validation, and test datasets
def get_data_loaders(data_dir, test_dir, batch_size):
    train_data = datasets.ImageFolder(os.path.join(data_dir), transform=transform)
    test_data = datasets.ImageFolder(os.path.join(test_dir), transform=test_transform)

    train_size = int(0.8 * len(train_data))
    val_size = len(train_data) - train_size
    train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, val_loader, test_loader

# Load pre-trained PyTorch models
model1 = torch.hub.load('pytorch/vision:v0.10.0', 'densenet169', pretrained=True)
model2 = torch.hub.load('pytorch/vision:v0.10.0', 'densenet201', pretrained=True)
model3 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model4 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model5 = timm.create_model('mobilenetv2_140', pretrained=True)
model6 = timm.create_model('efficientnet_b5', pretrained=True)

# Modify the models for binary classification
import torch
import torch.nn as nn
import torch.nn.init as init

# Assuming 'model' is your existing model
# Freeze all parameters in the existing model
for param in model1.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model1.classifier.in_features

# Modify the fc layer and add more layers for reducing val loss
model1.classifier = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model1 = model1.to(device)


import torch
import torch.nn as nn
import torch.nn.init as init

# Assuming 'model' is your existing model
# Freeze all parameters in the existing model
for param in model2.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model2.classifier.in_features

# Modify the fc layer and add more layers for reducing val loss
model2.classifier = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model2 = model2.to(device)



for param in model3.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model3.fc.in_features

# Modify the fc layer and add more layers for reducing val loss
model3.fc = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model3 = model3.to(device)

for param in model4.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model4.fc.in_features

# Modify the fc layer and add more layers for reducing val loss
model4.fc = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model4 = model4.to(device)

for param in model5.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model5.classifier.in_features

# Modify the fc layer and add more layers for reducing val loss
model5.classifier = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model5 = model5.to(device)

for param in model6.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model6.classifier.in_features

# Modify the fc layer and add more layers for reducing val loss
model6.classifier = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model6 = model6.to(device)

# Step 5: Load pre-trained PyTorch models
model1_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_DenseNet169.pth"
model2_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_DenseNet201.pth"
model3_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_ResNet101.pth"
model4_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_ResNet152.pth"
model5_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_MobileNetV2.pth"
model6_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_EfficientNet.pth"



model1.load_state_dict(torch.load(model1_path))
model2.load_state_dict(torch.load(model2_path))
model3.load_state_dict(torch.load(model3_path))
model4.load_state_dict(torch.load(model4_path))
model5.load_state_dict(torch.load(model5_path))
model6.load_state_dict(torch.load(model6_path))


# Set models to evaluation mode
model1.eval()
model2.eval()
model3.eval()
model4.eval()
model5.eval()
model6.eval()

# Get data loaders
train_loader, val_loader, test_loader = get_data_loaders("/kaggle/input/bavumub-shortwork/trainFinal/", "/kaggle/input/augmented-20x-10x/20x/20x/test", batch_size=128)

# Extract Convolutional Features Function
def extract_conv_features(model, dataloader):
    model.eval()
    features_list = []
    with torch.no_grad():
        for images, _ in dataloader:
            images = images.to(device)
            features = model(images).detach().cpu().numpy()
            features_list.append(features.reshape(features.shape[0], -1))
    return np.concatenate(features_list)

# Extract convolutional features from models
conv_features1 = extract_conv_features(model1, train_loader)
conv_features2 = extract_conv_features(model2, train_loader)
conv_features3 = extract_conv_features(model3, train_loader)
conv_features4 = extract_conv_features(model4, train_loader)
conv_features5 = extract_conv_features(model5, train_loader)
conv_features6 = extract_conv_features(model6, train_loader)

# Apply ELM on convolutional features
elm_models = [ELMClassifier() for _ in range(6)]
for elm_model, conv_features in zip(elm_models, [conv_features1, conv_features2, conv_features3, conv_features4, conv_features5, conv_features6]):
    elm_model.fit(conv_features, np.concatenate(all_targets))

# Get ground truth labels and make predictions using each ELM model
val_targets = []
elm_val_predictions = []

for images, targets in val_loader:
    images = images.to(device)
    targets = targets.numpy()
    val_targets.extend(targets)

    # Make predictions using each ELM model
    elm_model_predictions = []
    for elm_model, model in zip(elm_models, [model1, model2, model3, model4, model5, model6]):
        all_outputs = []
        with torch.no_grad():
            outputs = model(images).detach().cpu().numpy()
            all_outputs.append(outputs.reshape(outputs.shape[0], -1))

        predictions = elm_model.predict(np.concatenate(all_outputs))
        elm_model_predictions.append(predictions)

    elm_val_predictions.append(majority_voting(elm_model_predictions))

# Flatten the list of predictions
elm_majority_voting_val_predictions = [item for sublist in elm_val_predictions for item in sublist]

# Combine predictions using majority voting
def majority_voting(predictions_list):
    final_predictions = []
    for i in range(len(predictions_list[0])):
        preds = [predictions[i] for predictions in predictions_list]
        final_predictions.append(np.bincount(preds).argmax())
    return final_predictions

# Calculate accuracy
elm_val_accuracy = accuracy_score(val_targets, elm_majority_voting_val_predictions)
print("ELM ensemble validation accuracy:", elm_val_accuracy)

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets
from sklearn.metrics import accuracy_score
import numpy as np

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import numpy as np

class ELMClassifier:
    def __init__(self, n_hidden_units=100, n_output_neurons=2):
        self.n_hidden_units = n_hidden_units
        self.n_output_neurons = n_output_neurons

    def fit(self, X, y):
        # Normalize features to be in the range [-1, 1]
        X = 2 * (X - np.min(X)) / (np.max(X) - np.min(X)) - 1

        self.input_weights = np.random.normal(size=(X.shape[1], self.n_hidden_units))
        self.bias = np.random.normal(size=(self.n_hidden_units,))
        hidden_activations = np.dot(X, self.input_weights) + self.bias

        # Moore-Penrose pseudo-inverse to calculate output weights
        self.output_weights = np.linalg.pinv(hidden_activations).dot(y)

    def predict(self, X):
        # Normalize features to be in the range [-1, 1]
        X = 2 * (X - np.min(X)) / (np.max(X) - np.min(X)) - 1

        hidden_activations = np.dot(X, self.input_weights) + self.bias
        output = np.dot(hidden_activations, self.output_weights)
        return np.argmax(output, axis=1)

# Set random seed for reproducibility
seed = 12
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Define transformations for custom dataset
transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.10),
    T.RandomAffine(degrees=(-30, 30), translate=(0.2, 0.2), scale=(0.8, 1.2), shear=(-10, 10)),
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Define function to get data loaders for train, validation, and test datasets
def get_data_loaders(data_dir, test_dir, batch_size):
    train_data = datasets.ImageFolder(os.path.join(data_dir), transform=transform)
    test_data = datasets.ImageFolder(os.path.join(test_dir), transform=test_transform)

    train_size = int(0.8 * len(train_data))
    val_size = len(train_data) - train_size
    train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, val_loader, test_loader

import timm
# Load pre-trained PyTorch models
#model1 = torch.hub.load('pytorch/vision:v0.10.0', 'densenet169', pretrained=True)
model2 = torch.hub.load('pytorch/vision:v0.10.0', 'densenet201', pretrained=True)
model3 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model4 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)

import torch
import torch.nn as nn
import torch.nn.init as init

# Assuming 'model' is your existing model
# Freeze all parameters in the existing model
for param in model2.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model2.classifier.in_features

# Modify the fc layer and add more layers for reducing val loss
model2.classifier = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model2 = model2.to(device)

for param in model3.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model3.fc.in_features

# Modify the fc layer and add more layers for reducing val loss
model3.fc = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model3 = model3.to(device)

for param in model4.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model4.fc.in_features

# Modify the fc layer and add more layers for reducing val loss
model4.fc = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)  # Softmax activation function
)

model4 = model4.to(device)

# Step 5: Load pre-trained PyTorch models
#model1_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_DenseNet169.pth"
model2_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_DenseNet201.pth"
model3_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_ResNet101.pth"
model4_path = "/kaggle/input/cnn-bvm/CNN_Models/PTH/model_ResNet152.pth"

#model1.load_state_dict(torch.load(model1_path))
model2.load_state_dict(torch.load(model2_path))
model3.load_state_dict(torch.load(model3_path))
model4.load_state_dict(torch.load(model4_path))

# Set models to evaluation mode
#model1.eval()
model2.eval()
model3.eval()
model4.eval()

# Get data loaders
train_loader, val_loader, test_loader = get_data_loaders("/kaggle/input/bavumub-shortwork/trainFinal/", "/kaggle/input/augmented-20x-10x/20x/20x/test", batch_size=128)
# Apply ELM on each model's outputs
elm_models = [ELMClassifier() for _ in range(3)]

# Calculate the total train size using train_loader instead of train_dataset
total_train_size = len(train_loader.dataset)
split_size = total_train_size // 4

# Define start and end indices for each part
start_indices = [i * split_size for i in range(4)]
end_indices = [(i + 1) * split_size for i in range(4)]
end_indices[-1] = total_train_size  # Adjust the last end index

for elm_model, model, start_idx, end_idx in zip(elm_models, [model2, model3, model4], start_indices, end_indices):
    model.to(device)
    model.eval()

    # Create a data loader for the current model's portion of the dataset
    train_subset = torch.utils.data.Subset(train_loader.dataset, list(range(start_idx, end_idx)))
    train_loader_model = DataLoader(train_subset, batch_size=128, shuffle=True, num_workers=4)

    all_outputs = []
    all_targets = []

    for images, targets in train_loader_model:
        images = images.to(device)
        targets = targets.to(device)

        with torch.no_grad():
            outputs = model(images)
            all_outputs.append(outputs.detach().cpu().numpy().reshape(outputs.shape[0], -1))
            all_targets.append(targets.cpu().numpy().reshape(-1, 1))

    # Fit ELM with the outputs from the current model's data loader
    elm_model.fit(np.concatenate(all_outputs),
                  np.concatenate(all_targets))

# Get ground truth labels for validation set
val_targets = []
for _, targets in test_loader:
    val_targets.extend(targets.numpy())

# Make predictions using each ELM model
elm_val_predictions = []
for elm_model, model in zip(elm_models, [model2, model3, model4]):
    all_outputs = []
    for images, _ in test_loader:
        images = images.to(device)
        with torch.no_grad():
            outputs = model(images).detach().cpu().numpy()
            all_outputs.append(outputs.reshape(outputs.shape[0], -1))

    predictions = elm_model.predict(np.concatenate(all_outputs))
    elm_val_predictions.append(predictions)

# Combine predictions using majority voting
def majority_voting(predictions_list):
    final_predictions = []
    for preds in zip(*predictions_list):
        final_predictions.append(max(set(preds), key=preds.count))
    return final_predictions

elm_majority_voting_val_predictions = majority_voting(elm_val_predictions)

# Calculate accuracy
elm_val_accuracy = accuracy_score(val_targets, elm_majority_voting_val_predictions)
print("ELM ensemble validation accuracy:", elm_val_accuracy)